<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Ejercicios creados a partir de 2019-2 por Equipo Docente IIC2233. </font>
<font size='1'> Actualizados el 2023-2.</font>
</p>


# Ejercicios propuestos: Networking
## Networking con Python TCP

Los siguientes problemas se proveen como oportunidad de ejercitar los conceptos revisados en el material de **Networking**. Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una *issue* en el [foro del curso](https://github.com/IIC2233/syllabus/issues).

### Ejercicio 1: HTTP Requests y Perritos!

En este ejercicio deberás comunicarte con un servidor en internet! Usaremos la API [dog.ceo](https://dog.ceo/dog-api/) para descargar una foto aleatoria de un perrito en nuestro computador! Para conectarse a páginas web, es común que se usen los protocolos `HTTP` y `HTTPS`. En general las páginas web requieren canales seguros para la comunicación, por lo que usaremos `HTTPS`. Tu trabajo será comunicarte con el servidor en la nube usando **sockets** y *HTTP Requests*, y recibir e interpretar las *HTTP Responses* que nos mandará de vuelta, para obtener nuestra preciada foto de un perrito!

Para trabajar usando `HTTP` Requests, se te entregan las siguientes funciones: 
- `extraer_http_headers`: Recibe un **bytestring** que contiene los headers de la *HTTP request/response*, y retorna un diccionario con los pares `header` : `contenidos`. No es necesario que la apliques, esta se usa dentro de las siguientes funciones.
- `generar_http_request`: Recibe tres argumentos: El host al cual contactar, y el path al cual acceder. Por ejemplo, si [https://github.com/IIC2233/syllabus/issues](https://github.com/IIC2233/syllabus/issues) es la URL, el host corresponde a `github.com` y el path corresponde a `IIC2233/syllabus/issues`. Retorna un **string** con la *HTTP Request* formada.
- `recibir_http_response`: Recibe un **socket** como argumento. Recibe una *HTTP Response* desde el host. Esta función retorna un **bytestring**, que contiene el mensaje enviado desde el servidor.

A continuación, se muestran las funciones descritas:

In [1]:
# No debes modificar este código, pero eres bienvenido a revisarlo para ver como funciona
import json
from time import sleep

def generar_http_request(host, path="", method="GET"):
    """
    Retorna una string con formato de HTTP Request, lista para ser enviada.
    Recuerda que las HTTP requests tienen el siguiente formato:
    ```
    GET /path HTTP/1.1
    Host: direccion_host
    ```
    """
    request = f"{method} /{path} HTTP/1.1\r\n"
    request += f"Host: {host}\r\n\r\n"
    return request


def recibir_http_response(sock):
    """
    Retorna una bytestring con la HTTP Response recibida del host.
    Las HTTP response tienen un formato similar:
    ```
    GET /path HTTP/1.1
    Host: direccion_host
    Content-Length: 12356789
    Otro-Header: ...

    Mensaje recibido...
    ```
    Notar que luego de una línea en blanco, viene el mensaje recibido. Este puede ser cualquier
    secuencia de bytes, como texto, un JSON, o una imagen.
    ESTA FUNCIÓN RETORNA DICHO MENSAJE RECIBIDO
    """
    buffer = sock.recv(2024) # Este primer buffer contiene parte headers y parte datos

    headers, datos = buffer.split(b'\r\n\r\n') # una línea en blanco indica su separación
    
    if b'\r\n' in datos:
        datos = datos.split(b'\r\n')[1]
        
    dict_headers = extraer_http_headers(headers)
    
    # El tamaño en bytes del mensaje está en el header Content-Length
    if "Content-Length" in dict_headers:
        tamano_mensaje = int(dict_headers['Content-Length'])
    else:
        tamano_mensaje = len(datos)

    tamano_descargado = len(datos) # Ya descargamos la primera parte (venía junto con los headers)
    print(f"El tamaño del mensaje es de {tamano_mensaje} bytes y tenemos {tamano_descargado} datos")
    # A continuación, se descarga el resto
    if tamano_descargado < tamano_mensaje:
        print("Descargando datos ....")
    while tamano_descargado < tamano_mensaje:
        print("Descargando... {:3d}%".format(round(100*tamano_descargado/tamano_mensaje)), end="\r")
        buffer = sock.recv(2024)
        datos += buffer
        tamano_descargado += len(buffer)
        sleep(0.2)
    print("El mensaje ha sido recibido!")
    return datos


def extraer_http_headers(headers_string):
    """
    Transforma un string que contiene los headers de un http response (headers_string)
    a un diccionario, el cual contiene pares de la forma { nombre_header : datos }
    Retorna el diccionario descrito anteriormente.
    """
    headers_list = headers_string.split(b'\r\n')
    protocol, code, code_name = headers_list[0].split(b' ')
    headers = {'protocol': protocol, 'code': code, 'code_name': code_name}
    for header in headers_list[1:]:
        name, data = header.split(b': ')
        headers[name.decode()] = data.decode()
    return headers

Ahora, debes completar el siguiente código, aplicando la materia vista en los `notebooks`. Es muy recomendable que uses los comentarios para guiarte.

In [ ]:
# Release
import ssl # Permite establecer una conexión segura para el protocolo HTTPS
import socket
import json


if __name__ == "__main__":
    host = "dog.ceo" # Este es el host que usaremos. En general puede ser una IP o una URL
    path = "api/breeds/image/random" # Ruta del host a la cual queremos acceder
    # COMPLETAR (¿Cuál es el puerto que se usa para una conexión HTTPS?)
    port = 

    # Crea un socket en la variable socket_https
    # Necesitamos que la conexión sea usando IPv4, y queremos hacerla a través de TCP
    socket_https = 

    # Estas líneas hacen que la conexión funcione sobre HTTPS, no debes modificarlas
    context = ssl.SSLContext(ssl.PROTOCOL_TLS)
    socket_https = context.wrap_socket(socket_https, server_hostname=host)
    socket_https.settimeout(10) # Obliga al socket a lanzar una excepción al pasar N segundos

    try:
        # Para las siguientes partes, recuera que tienes las funciones 
        # generar_http_request y recibir_http_response entregadas anteriormente
        # COMPLETAR
        # CONECTAR SOCKET AQUÍ (El socket a utilizar sigue en la variable socket_https)
        
        
        # GENERAR Y ENVIAR HTTP REQUEST AQUÍ
        
        
        
        # RECIBIR HTTP RESPONSE AQUÍ
        
        
        # La respuesta debería ser un JSON, por lo que debes decodificarla a continuación
        json_obj = 
        
    except ConnectionError as err:
        print("Ocurrió un error")
    finally:
        # CERRAR EL SOCKET UTILIZADO AQUÍ
        
        

    if json_obj['status'] != "success":
        print("Ha ocurrido un error!")
        exit(1)

    # El JSON recibido contiene una nueva url que usaremos
    url = json_obj['message']
    host, path = url[8:].split("/", 1)
    print(f"Nueva URL: {url}\nNuevo host: {host}\nNuevo path: {path}")

    # Debido a que el host es distinto, debemos abrir un nuevo socket
    # COMPLETAR
    # Crea un socket en la variable socket_https, de manera análoga a la anterior
    
    
    # Nuevamente, estas líneas hacen que la conexión funcione sobre HTTPS, no debes modificarlas
    context = ssl.SSLContext(ssl.PROTOCOL_TLSv1)
    socket_https = context.wrap_socket(socket_https, server_hostname=host)
    socket_https.settimeout(10) # Obliga al socket a lanzar una excepción al pasar N segundos

    try:
        # COMPLETAR
        # CONECTAR SOCKET AQUÍ
        
        
        # GENERAR Y ENVIAR HTTP REQUEST AQUÍ
        
        
        
        # RECIBIR HTTP RESPONSE AQUÍ
        

        # Finalmente, guardamos los datos recibidos en un archivo con formato jpg
        with open('perrito.jpg', 'wb') as image_file:
            image_file.write(response)
            print("La imagen se ha guardado en perrito.jpg !")
    except ConnectionError as err:
        print("Ocurrió un error")
    finally:
        # CERRAR EL SOCKET UTILIZADO AQUÍ
        
        